## ***<h1>Credit Card Fraud Detection using Single Feed-Forward Multi-Layer Perceptron ANN***

***<h2>Phase 1: Importing required libraries***

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler        # for scaling and normalising to train ANN effectively
from sklearn.model_selection import train_test_split    # for splitting data
from sklearn.utils import class_weight                  # helps handle class imbalance by assigning weights

# for model performance evaluation
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, precision_recall_curve, auc  

from tensorflow.keras.models import Sequential          # Sequential model type (layers stacked one after another)
from tensorflow.keras.layers import Dense, Dropout      # Dense = fully connected layer, Dropout = regularization
from tensorflow.keras.optimizers import Adam            # Optimizer (gradient descent variant) for training

import warnings
warnings.filterwarnings('ignore')                       # suppressing warnings for clean output

# Set plot style
sns.set(style="darkgrid")

***<h2>Phase 2: Loading the dataset to work on***

In [ ]:
df = pd.read_csv("creditcard.csv")                                    # Loading dataset and understanding it

print("Shape of the dataset:", df.shape)                              # prints shape of the dataset(rows and columns)

print("\nDistribution of Class:\n", df['Class'].value_counts())       # distribution of legit and fraud classes

print("\nMissing values:\n", df.isnull().sum().sum())                 # values missing in the entire dataset

In [ ]:
print("\nFirst five rows of the dataset:\n")                          # first 5 rows
df.head()

In [ ]:
print("\nLast five rows of the dataset:\n")                           # last 5 rows
df.tail()

***<h2>Phase 3 : EDA***

In [ ]:
# Class Imbalance visualization - shows fraud vs non-fraud counts.
# Visualizes the severe class imbalance, showing far fewer fraud cases compared to legitimate ones.

plt.figure(figsize=(6,4))
sns.countplot(x='Class', data=df)
plt.title("Fraud vs Non-Fraud Transactions")
plt.show()

fraud_cases = df[df['Class']==1]
legit_cases = df[df['Class']==0]

print("Fraud cases:", len(fraud_cases))
print("Legit cases:", len(legit_cases))
print("Fraud percentage: {:.4f}%".format(len(fraud_cases)/len(df)*100))

In [ ]:
# Fraud vs Non-Fraud transaction amounts, highlighting differences and outliers.

plt.figure(figsize=(8,5))
sns.boxplot(x="Class", y="Amount", data=df)
plt.title("Amount by Transaction Class")
plt.show()

In [ ]:
# Heatmap showing correlations between features to identify relationships and multicollinearity.  

plt.figure(figsize=(12,8))
sns.heatmap(df.corr(), cmap="BuPu", center=0, linewidths=0.1) 
plt.title("Correlation Heatmap of Features")
plt.show()

In [ ]:
# KDE plots showing distribution of PCA components (V1–V4) for fraud(1) vs non-fraud(0) to highlight feature separation.  

plt.figure(figsize=(12,6))
for i, col in enumerate(['V1','V2','V3','V4']):
    plt.subplot(2,2,i+1)
    sns.kdeplot(data=df, x=col, hue='Class', fill=True, common_norm=False, alpha=0.5, palette=['skyblue', 'purple']) 
    plt.title(f"Distribution of {col} by Class")
plt.tight_layout()
plt.show()

***<h2>Phase 4: Preprocessing***

In [ ]:
# Standardizes the "Amount" feature to improve model training efficiency and stability.  
# Drops "Time" and raw "Amount" since they do not contribute significantly after scaling.  

scaler = StandardScaler()
df['Amount_scaled'] = scaler.fit_transform(df[['Amount']])
df = df.drop(['Time','Amount'], axis=1)

In [ ]:
# Separates dataset into features (X) for model input and target (y) representing fraud vs non-fraud.  
# This prepares the data for training and evaluation of the classification model.  

X = df.drop('Class', axis=1)
y = df['Class']

In [ ]:
# Splits the dataset into training (80%) and testing (20%) sets while preserving the fraud-to-legit ratio using stratify.  
# Ensures the model sees balanced class proportions during training and evaluation, preventing bias.  
# Important: Stratification maintains the rare fraud cases in both train and test sets for reliable performance assessment.  

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

print("Training Data:", X_train.shape, "Testing Data:", X_test.shape,"\n")
print("Fraud % in train:", y_train.mean()*100, "\nFraud % in test:", y_test.mean()*100)

In [ ]:
# Convert y_train to a 1D NumPy array(if required) to avoid shape issues during class weight computation.  
y_train = np.array(y_train).ravel()

# Computes class weights automatically to handle severe imbalance between fraud (1) and non-fraud (0).  
class_weights = class_weight.compute_class_weight(
    class_weight = 'balanced',
    classes = np.unique(y_train),                # Must match the unique labels in y_train
    y = y_train
)

# Generates a dictionary mapping each class (0 and 1) to its weight, giving more importance to rare fraud cases.
class_weights_dict = dict(zip(np.unique(y_train), class_weights))
print("Class Weights:\n", class_weights_dict)

# Important: These weights are later passed to model training so that the ANN does not ignore minority fraud samples.

***<h2>Phase 5: ANN model training***

In [ ]:
# Builds the Artificial Neural Network (ANN) using a Sequential model with stacked layers.  
# Input Layer: Takes features from training data, first Dense layer has 32 neurons with ReLU activation.  
# Dropout Layers: Randomly drop 30% of neurons during training to reduce overfitting.  
# Hidden Layer: Second Dense layer with 16 neurons and ReLU activation for deeper feature learning.  
# Output Layer: Single neuron with sigmoid activation outputs probability of fraud (1) or non-fraud (0).  

model = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(16, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

# Compiles the model using Adam optimizer, binary crossentropy loss (for classification), and accuracy as the metric.  
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
# Trains the ANN model on training data with class weights applied to handle imbalance (fraud vs non-fraud).  
# Uses 20 epochs and large batch size (2048) for efficient training on big data.  
# validation_split = 0.2 reserves 20% of training data for validation to monitor overfitting during training.  
# class_weight ensures the rare fraud cases are given higher importance so the model learns to detect them.  
# verbose = 1 shows detailed progress of training (loss and accuracy per epoch).  

history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=2048,
    validation_split=0.2,
    class_weight=class_weights_dict,
    verbose=1 )

# Saves the trained model as 'fraud_model.keras' so it can be reused later without retraining.  
model.save("fraud_model.keras")

***<h2>Phase 6: Model Evaluation***

In [ ]:
# Predictions on test data (probabilities between 0 and 1)
y_pred_prob = model.predict(X_test)

# Convert predicted probabilities into binary classes (0 or 1) using threshold = 0.5
y_pred = (y_pred_prob > 0.5).astype(int)

# Classification Report - Print Precision, Recall, F1-score for both classes
print("Classification Report:\n", classification_report(y_test, y_pred))

#Calculate ROC-AUC score (area under ROC curve)
roc_auc = roc_auc_score(y_test, y_pred_prob)
print("ROC-AUC Score:", roc_auc)

# Important: threshold can be tuned (e.g., 0.3 or 0.7) to improve fraud detection
#    -Recall for fraud class (1) is most critical since we must catch maximum fraud cases
#    -AUC is a strong metric for imbalanced data, measures fraud vs non-fraud separability- 
#    -Display the ROC-AUC score (closer to 1 = better classification performance)

In [ ]:
# Compute and visualise confusion matrix (rows = actual labels, columns = predicted labels)

cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(5,4))

custom_cmap = sns.blend_palette(['skyblue', 'purple'], as_cmap=True) 
sns.heatmap(cm, annot=True, fmt='d', cmap=custom_cmap, edgecolor= 'black')
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

In [ ]:
# Compute and visualise PR AUC, precision and recall values at different probability thresholds

prec, recall, _ = precision_recall_curve(y_test, y_pred_prob)
pr_auc = auc(recall, prec)
plt.plot(recall, prec, label=f'PR curve (AUC = {pr_auc:.4f})', color='purple')
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve")
plt.legend()
plt.show()

In [ ]:
# Simulated test labels and predicted probabilities

np.random.seed(42)
y_test_roc = np.random.randint(0, 2, 200)         
y_pred_prob_roc = np.random.rand(200)            

# Compute ROC curve and AUC
fpr, tpr, thresholds = roc_curve(y_test_roc, y_pred_prob_roc)
roc_auc = auc(fpr, tpr)

# Plot ROC Curve
plt.figure(figsize=(8,6))
plt.plot(fpr, tpr, color='purple', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})') 
plt.plot([0,1], [0,1], color='skyblue', lw=2, linestyle='--')                          
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()

<i>ROC Curve evaluates model performance by comparing True Positive Rate (TPR) vs False Positive Rate (FPR).  
Important: TPR = ability to correctly identify fraud cases, FPR = incorrectly flagging legit cases as fraud.  

<i>AUC (Area Under Curve) shows overall separability: closer to 1 = excellent model, 0.5 = random guessing.  
Important: In imbalanced datasets (like fraud detection), ROC-AUC can sometimes look good even if fraud detection is poor.  

<i>The baseline diagonal line represents a random classifier (no predictive power).  
Important: A good model’s ROC curve should lie far above this diagonal.  

<i>Practical note: For fraud detection, Precision-Recall Curve is often more informative than ROC, since it focuses on how well the model captures rare fraud cases without too many false alarms. ROC Curve evaluates model performance by comparing True Positive Rate (TPR) vs False Positive Rate (FPR).</i>

In [ ]:
# Training Loss & Accuracy Curves

plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history.history['loss'], label='Train Loss', color='purple') 
plt.plot(history.history['val_loss'], label='Val Loss', color='skyblue') 
plt.legend(); plt.title("Loss Over Epochs")

plt.subplot(1,2,2)
plt.plot(history.history['accuracy'], label='Train Acc', color='purple') 
plt.plot(history.history['val_accuracy'], label='Val Acc', color='skyblue') 
plt.legend(); plt.title("Accuracy Over Epochs")

plt.show()

<i>Plots training vs validation loss across epochs to check how well the model is learning.
Important: If validation loss increases while training loss decreases → model is overfitting.

<i>Plots training vs validation accuracy across epochs to evaluate performance.
Important: Both curves should ideally converge; large gaps indicate overfitting or underfitting.

<i>Loss curve is more reliable than accuracy for imbalanced data (fraud detection),
since accuracy can be misleading if the model predicts mostly non-fraud.

<i>These plots help in tuning epochs, batch size, and dropout rate to improve generalization.

In [ ]:
# Function to test model with new unseen transaction
# input_data: list of 30 features (V1-V28, Amount_scaled, etc.) Example: a single row transaction

def predict_transaction(model, input_data):

    # Converts the list into a NumPy array and reshapes it to match the model’s input format.
    input_array = np.array(input_data).reshape(1,-1)    
    
    prediction_prob = model.predict(input_array)[0][0]
    prediction = int(prediction_prob > 0.5)
    print("Prediction Probability:", prediction_prob)
    print("Transaction Classified as:", "FRAUD" if prediction==1 else "LEGITIMATE")

In [ ]:
# Example Usage:
# Taking first test transaction
sample = X_test.iloc[0].values
predict_transaction(model, sample)

In [ ]:
# Sample 1: Legitimate Transaction (random values from test set)

sample_legit = X_test[y_test==0].iloc[0].values
print("Testing with a LEGITIMATE transaction\n\n")
predict_transaction(model, sample_legit)

In [ ]:
# Sample 2: Fraudulent Transaction (random values from test set)

sample_fraud = X_test[y_test==1].iloc[0].values
print("\nTesting with a FRAUD transaction\n\n")
predict_transaction(model, sample_fraud)